<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/AI_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI')

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import copy
import json
from openai import OpenAI
from openai import ChatCompletion

#from utils_local import o1_tools

client = OpenAI(api_key=openai_api_key)
#O1_MODEL = 'o1-mini'
GPT_MODEL = 'gpt-4o-mini'

In [ ]:
# System prompt for the execution model
gpt4o_system_prompt = """
You are a helpful data analyst responsible for analysis provided data file using the tools provided.
You must explain your decision-making process across various steps. if ask to read data, you should read the
data file and using the provided tool and summurize the infomation in in the first 5 rows.  If you are not asked to read
data, you should answer a user question to the best of your ability.

"""

In [ ]:
tool_metadata = {
    "name": "read_csv_to_dataframe",
    "description": "Reads a CSV file and returns the DataFrame.",
    "parameters": {
        "type": "object",
        "properties": {
            "file_path": {"type": "string", "description": "Path to the CSV file."}
        },
        "required": ["file_path"]
    }
}

In [ ]:
tools = [tool_metadata]

In [ ]:
# Tools
def read_csv_to_dataframe(file_path: str) -> str:
    import pandas as pd
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        return f"Error reading CSV file: {e}"

In [ ]:
# Map function names to actual functions
function_mapping = {
    'read_csv_to_dataframe': read_csv_to_dataframe
}

In [ ]:
data = '/content/Beer_small_test.csv'
messages = [
    {"role": "system", "content": gpt4o_system_prompt},
    {"role": "user", "content": f"Can you read the {data} and tell me what it is about?"}
]
# User query
response = ChatCompletion.create(
    model="gpt-4o",
    messages=messages,
    functions=tools,
    function_call={"name": "read_csv_to_dataframe"}
)

In [ ]:
def call_gpt4o(plan):
    gpt4o_policy_prompt = gpt4o_system_prompt.replace("{policy}", plan)
    messages = [
        {'role': 'system', 'content': gpt4o_policy_prompt},
    ]

    while True:
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            tools=TOOLS,
            parallel_tool_calls=False
        )

        assistant_message = response.choices[0].message.to_dict()
        print(assistant_message)
        messages.append(assistant_message)

        append_message({'type': 'assistant', 'content': assistant_message.get('content', '')})

        if (response.choices[0].message.tool_calls and
            response.choices[0].message.tool_calls[0].function.name == 'instructions_complete'):
            break

        if not response.choices[0].message.tool_calls:
            continue

        for tool in response.choices[0].message.tool_calls:
            tool_id = tool.id
            function_name = tool.function.name
            input_arguments_str = tool.function.arguments

            append_message({'type': 'tool_call', 'function_name': function_name, 'arguments': input_arguments_str})

            try:
                input_arguments = json.loads(input_arguments_str)
            except (ValueError, json.JSONDecodeError):
                continue

            if function_name in function_mapping:
                try:
                    function_response = function_mapping[function_name](**input_arguments)
                except Exception as e:
                    function_response = {'error': str(e)}
            else:
                function_response = {'error': f"Function '{function_name}' not implemented."}

            try:
                serialized_output = json.dumps(function_response)
            except (TypeError, ValueError):
                serialized_output = str(function_response)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_id,
                "content": serialized_output
            })

            append_message({'type': 'tool_response', 'function_name': function_name, 'response': serialized_output})

    return messages